In [1]:
import time
from selenium import webdriver
from pprint import pprint
import concurrent
import pandas as pd

In [2]:
from selenium.webdriver.chrome.options import Options
chrome_options = Options()
chrome_options.add_argument("--disable-extensions")
chrome_prefs = {
      'profile.managed_default_content_settings.images': 2,
    }
chrome_options.add_experimental_option('prefs', chrome_prefs)

In [3]:
driver = webdriver.Chrome(options=chrome_options)
driver.get('https://www.wandoujia.com/category/game');

In [4]:
category_href = {}

categories = driver.find_elements_by_class_name('cate-link')
for c in categories:
    name = c.text
    href = c.get_attribute('href')
    category_href[name] = href

child_cats = driver.find_elements_by_class_name('child-cate')
for c in child_cats:
    children = c.find_elements_by_tag_name('a')
    for child in children:
        name = child.text
        href = child.get_attribute('href')
        category_href[name] = href

driver.close()

In [5]:
category_href

{'休闲益智': 'https://www.wandoujia.com/category/6001',
 '跑酷竞速': 'https://www.wandoujia.com/category/6003',
 '扑克棋牌': 'https://www.wandoujia.com/category/6008',
 '动作冒险': 'https://www.wandoujia.com/category/6004',
 '飞行射击': 'https://www.wandoujia.com/category/6002',
 '经营策略': 'https://www.wandoujia.com/category/6007',
 '网络游戏': 'https://www.wandoujia.com/category/6009',
 '体育竞技': 'https://www.wandoujia.com/category/6005',
 '角色扮演': 'https://www.wandoujia.com/category/6004_926',
 '辅助工具': 'https://www.wandoujia.com/category/5015',
 '休闲': 'https://www.wandoujia.com/category/6001_666',
 '消除': 'https://www.wandoujia.com/category/6001_668',
 '音乐': 'https://www.wandoujia.com/category/6001_670',
 '益智': 'https://www.wandoujia.com/category/6001_672',
 '捕鱼': 'https://www.wandoujia.com/category/6001_723',
 '解谜': 'https://www.wandoujia.com/category/6001_755',
 '跑酷': 'https://www.wandoujia.com/category/6003_677',
 '赛车': 'https://www.wandoujia.com/category/6003_678',
 '摩托': 'https://www.wandoujia.com/category/6

In [6]:
def crawl_category(category, url, output):
    driver = webdriver.Chrome(options=chrome_options)
    driver.get(url)
    
    # load more until theres no more
    try:
        while True:
            driver.find_element_by_class_name('refresh-btn').click()
    except:
        print(f'{category} Done loading more')
        
    cards = driver.find_elements_by_class_name('card')
    print(f'{category} found {len(cards)} cards')T
    
    for i, card in enumerate(cards):
        data_pn = card.get_attribute('data-pn')
        href = card.find_element_by_class_name('app-desc')\
                   .find_element_by_class_name('name')\
                   .get_attribute('href')

        if data_pn not in output:
            output[data_pn] = href
        
        if i % 100 == 0:
            print(f'{category} {i}')
            
    driver.close()

In [7]:
pn_href = {}

with concurrent.futures.ThreadPoolExecutor(max_workers=15) as executor:
    for cname in category_href:
        url = category_href[cname]
        executor.submit(crawl_category, cname, url, pn_href)

角色扮演 Done loading more
角色扮演 found 199 cards
角色扮演 0
辅助工具 Done loading more
辅助工具 found 290 cards
辅助工具 0
角色扮演 100
辅助工具 100
捕鱼 Done loading more
捕鱼 found 405 cards
捕鱼 0
益智 Done loading more
益智 found 559 cards
益智 0
捕鱼 100
辅助工具 200
音乐 Done loading more
音乐 found 616 cards
音乐 0
益智 100
捕鱼 200
音乐 100
益智 200
捕鱼 300
音乐 200
解谜 Done loading more
解谜 found 434 cards
解谜 0
体育竞技 Done loading more
体育竞技 found 933 cards
体育竞技 0
益智 300
休闲 Done loading more
休闲 found 737 cards
休闲 0
捕鱼 400
扑克棋牌 Done loading more
飞行射击 Done loading more
扑克棋牌 found 969 cards
网络游戏 Done loading more
扑克棋牌 0
消除 Done loading more
飞行射击 found 971 cards
飞行射击 0
消除 found 947 cards
网络游戏 found 939 cards
消除 0
网络游戏 0
经营策略 Done loading more
经营策略 found 982 cards
经营策略 0
动作冒险 Done loading more
动作冒险 found 969 cards
音乐 300
动作冒险 0
跑酷 Done loading more
跑酷 found 316 cards
跑酷 0
跑酷竞速 Done loading more
解谜 100
跑酷竞速 found 982 cards
跑酷竞速 0
休闲益智 Done loading more
休闲益智 found 953 cards
休闲益智 0
体育竞技 100
益智 400
休闲 100
扑克棋牌 100
飞行射击 100
消除 100
网络游戏 100
经营策略 100
跑酷 10

In [8]:
len(pn_href)

15816

In [9]:
pn_l_href = {pn: [pn_href[pn]] for pn in pn_href}
df = pd.DataFrame.from_dict(pn_l_href, orient='index', columns=['url'])

In [10]:
df

,url
com.miHoYo.bh3.wdj,https://www.wandoujia.com/apps/com.miHoYo.bh3.wdj
com.tencent.tmgp.dragonnest,https://www.wandoujia.com/apps/com.tencent.tmg...
com.tencent.tmgp.tstl,https://www.wandoujia.com/apps/com.tencent.tmg...
com.netease.gmdl.uc,https://www.wandoujia.com/apps/com.netease.gmd...
com.snailgame.jysgqy.aligames,https://www.wandoujia.com/apps/com.snailgame.j...
com.netease.ma71.aligames,https://www.wandoujia.com/apps/com.netease.ma7...
gamelab3d.com.FreeRoam3D,https://www.wandoujia.com/apps/gamelab3d.com.F...
com.orange.org_player_new_alone2662.uc,https://www.wandoujia.com/apps/com.orange.org_...
com.xmcy.hykb,https://www.wandoujia.com/apps/com.xmcy.hykb
com.xcpsp.zssbdyl.mt,https://www.wandoujia.com/apps/com.xcpsp.zssbd...


In [11]:
df.to_csv('game_data.csv')